<a href="https://colab.research.google.com/github/pakocica/Proofreader/blob/main/Correct_typos_latex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latex File Proofreader

This is a simple *Python* program that proofreads a provided *LaTeX* file and displays a list of suggested corrections. The program splits the provided file into chunks of chosen size and uses the [OpenAI API](https://platform.openai.com) to suggest corrections. For each corrected sentence, the program displays the diff of the original sentence and its suggested form.

## Instructions

1. Save your OpenAI API key in the secret named `api_key`.
2. Run all the utilities cells.
3. [Load the LaTeX file](#load-file).
4. Run the [main program](#interactive-form)

<details><summary>Click for Detailed Instructions</summary>

1. Save your OpenAI API key in the secret named `api_key`.  
    * Create an OpenAI API account if you don't have one.
    * Generate an OpenAI API key – click on `Create new secret key` in the [API keys page](https://platform.openai.com/api-keys).
    * Click on the key icon in the left Colab panel.
    * Insert the API key there (make sure to give the notebook access to `api_key` secret each time you start a session).

2. Run all the utilities cells.
    * Open the table of contents (left Colab panel)
    * Click on the three dots right from the section *Utilities* and click on `Run cells in section`.
    * This imports necessary libraries and defines functions used in the main program.

3. [Load the LaTeX file](#load-file).
    * Fill in the name of a local LaTeX file or the URL of the file (e.g., a Dropbox link) in [this form](#load-file), and run the corresponding cell.
    * Instructions to upload a file to the Colab session ([image](https://github.com/pakocica/Proofreader/blob/804925be9816d0503840813650f79ce442b32fb8/upload_file.png)):
        * Click on the folder icon in Colab's left panel.
        * You can drag-and-drop the file there, or click on the `Upload to session storage` icon.
        * Remember to re-upload the file for each session.

4. Run the [main program](#interactive-form)
</details>

### GPT Model Choice

* The `gpt-4o-mini` model is sufficient in most cases. Proofreading a 50-page document using it costs about \$0.05.
* The `gpt-4o` model provides slightly more precise results. Proofreading a 50-page document using it costs about \$0.5.
* **Recommendation:** Use `gpt-4o-mini` repeatedly to find the most obvious typos. In case of style improvements where understanding the context is important, use `gpt-4o`. Monitor the [actual usage](https://platform.openai.com/usage).

### Output

The output contains a widget with the following tabs:

* **Summary:** The list of recommended corrections with no other details.
* **With Explanations:** The list of corrections with explanations included.
* **Debug:** Corrections split into sub-categories **Details** and **Errors**. The corrections are categorized by sections (left tab) and contain other details useful for debugging, specifically the correction ID. The correction ID can be used to [inspect the details of the correction](#correction-details). Any correction for which either the *original sentence* was hallucinated or is the same as the *corrected sentence* is put into the error category.
* **Progress:** Updates about which part of the document is being processed.

### Comments

* The output can be copy-pasted to Gmail, but some of the formulas won't be formatted well.
* With a right click inside the widget window, you can print its contents.
* If you provide file *URL* instead of name of a local file, then make sure that the file gets downloaded indeed as a *LaTeX* document and not a *HTML*. (A link to the raw version, not *HTML* version, of the document must be provided.)
* If you want to edit individual functions, you can [turn on the debug mode](#debug-settings), or replace the variable `DEBUG` by `True` in the cells in which you want to run the example usage.

---

*This notebook was created by [Pavel Kocourek](https://www.pkocourek.com/).*

Associated GitHub Repository: [github.com/pakocica/Proofreader](https://github.com/pakocica/Proofreader).

---

# Utilities

Installs dependencies, import libraries, and defined utility functions that are used by the main program.

In [ ]:
#@title Debug Mode Settings
#@markdown <a name="debug-settings"></a>
#@markdown Choose if you want example usage of each cell to be run
#@markdown (run this cell after changing the settings):

DEBUG = False #@param {type:"boolean"}

In [ ]:
#@title Instal and import dependencies
!pip install parsimonious > install.log
!pip install openai tiktoken >> install.log

# !pip install click >> install.log # Y/N input

import re
from IPython.display import display, Markdown, HTML #, clear_output
from google.colab import widgets

from tqdm import tqdm

# obtaining the tex file
from urllib.parse import urlparse
import os
import requests

from openai import OpenAI
import tiktoken
import json
openai_client_cost = 0

from google.colab import userdata
client = OpenAI(api_key=userdata.get('api_key'))

In [ ]:
#@title Load the file to string (local file or URL)

def display_long_text(text):
    """
    Displays text in a window that can be scrolled.

    Args:
        text (str): The text to display.
    """
    display(HTML(f"""
    <div style="
        border: 1px solid #ccc;
        border-radius: 8px;
        padding: 15px;
        margin: 20px 0;
        box-shadow: 0 2px 8px;
        width: 80%;
        height: 200px;
        resize: both;
        overflow: auto;
    ">
        <pre style="white-space: pre-wrap;">{text}</pre>
    </div>
    """))


def load_file(file_path: str):
    """
    If a URL is provided, the file is downloaded, saved locally, and returned as str.
    Otherwise str of local file is given.

    Args:
        file_path (str): The path to the local file or URL.
         - has to be in the raw text format, in case of Dropbox, use '&dl=1' at the end of the link

    Returns:
        str: The contents of the file as a string.
    """

    # determine if file_name in case of url
    parsed_url = urlparse(file_path)
    is_url = parsed_url.scheme in ('http', 'https', 'ftp')

    # determine file name
    if is_url:
        # url
        file_name = os.path.basename(parsed_url.path)
    elif os.path.exists(file_path):
        # local file
        file_name = file_path
    else:
        raise ValueError(f'File not found: {file_path}')

    # download the file if url was provided
    if is_url:
        file_path = file_path.replace('&dl=0', '&dl=1')
        response = requests.get(file_path)

        if response.status_code != 200:
            raise ValueError(f"Error downloading file: {file_path}")
        else:
            # save the file locally
            with open(file_name, 'w') as f:
                f.write(response.text)

    # load the local file into a string
    with open(file_name, 'r') as f:
        return f.read()

# Example Use
if DEBUG:
    file_path = "https://raw.githubusercontent.com/pakocica/Proofreader/main/example.tex"
    example_text = load_file(file_path)
    display_long_text(example_text)


In [ ]:
#@title Remove comments and normalize the latex text
def remove_comments_from_line(line):
    # Use regex to remove comments, but keep the newline character
    return re.sub(r'(?<!\\)%.*', '', line)

def remove_clutter_from_latex(input: str):
    """
    Takes a latex text and:
        1. Removes comments
        2. Removes double space
        3. Removes tripple+ new lines
        4. Replaces special characters

    Returns whas is left
    """

    input_lines = input.split('\n')

    # Remove comments
    lines = [remove_comments_from_line(line) for line in input_lines if not line.startswith('%')]
    text_no_comments = '\n'.join(lines)

    # Remove extra space
    text_no_double_space = re.sub(r' {2,}', ' ', text_no_comments)

    # Remove extra empty lines
    output = re.sub(r'\n{3,}', '\n\n', text_no_double_space)

    # Replace special characters
    output = output.replace('`', "'").replace('’', "'")
    output = output.replace('``', '"').replace('’’', '"'). replace("''",'"')

    # Choose the math encoding for Markdown
    output = output.replace(r"\[", "$$").replace(r'\]', "$$")

    return output

# Example Usage

if DEBUG:
    cleaned_text = remove_clutter_from_latex(example_text)
    display_long_text(cleaned_text)

In [ ]:
#@title Split file by sections and subsections

def extract_braces_content(s):
    """
    Extracts what is inside the first matching pair of braces
    """
    stack = []
    start = -1
    for i, char in enumerate(s):
        if char == '{':
            if not stack:
                start = i + 1
            stack.append('{')
        elif char == '}':
            if stack:
                stack.pop()
                if not stack:
                    return s[start:i]
    return None

def file_to_sections(text: str):
    """
    Takes the provide latex file string and splits it by sections and subsections
    """

    text_clean = remove_clutter_from_latex(text)

    split_by_title = text_clean.split(r'\title')
    if len(split_by_title) > 1:
        title = extract_braces_content(split_by_title[1])
    else:
        title = 'Untitled'

    sections = text_clean.split(r'\section')
    sections = [sections[0]] + [r'\section' + s for s in sections[1:]]

    sections_sub = []
    for s in sections:
        subsections = s.split(r'\subsection')
        subsections = [subsections[0]] + [r'\subsection' + s for s in subsections[1:]]
        sections_sub.append(subsections)

    return title, sections, sections_sub

def display_contents(text:str):
    """
    Displays contents of the document
    """

    title, sections, sections_sub = file_to_sections(text)

    display(Markdown(f"### {title}"))

    tb = widgets.TabBar(['Contents'] + list(range(len(sections))), location='top')

    with tb.output_to(0, select=False):
        for i, s in enumerate(sections[1:]):
            name = extract_braces_content(s)
            display(Markdown(f'#### {i+1}. ' + name))

            for j, ss in enumerate(sections_sub[i][1:]):
                name = extract_braces_content(ss)
                display(Markdown(f'##### $\qquad\quad$ {i+1}.{j+1}. ' + name))

    for i, s in enumerate(sections):
        sub = sections_sub[i]
        with tb.output_to(i+1, select=False):
            if len(sub) == 1:
                print(s)
            else:
                tb_ss = widgets.TabBar(list(range(len(sub))), location='top')
                for j, ss in enumerate(sub):
                    with tb_ss.output_to(j, select=False):
                        print(ss)



# Example Usage

if DEBUG:
    display_contents(example_text)

In [ ]:
#@title Split into chunks

def split_latex_text(text, target_size=1600):
    """
    Split a LaTeX text into chunks of a given number of characters.
     - split by paragraphs

    Args:
        text (str): The LaTeX text to split.
        target_size (int): The target number of characters per chunk.

    Returns:
        list: A list of chunks of the LaTeX text.
    """

    # Split the text into sentences
    pieces = re.split(r'\n\n', text)
    pieces = [p + '\n\n' for p in pieces[:-1]] + [pieces[-1]]

    chunks = []
    current_chunk = ""
    for piece in pieces:

        if len(piece) == 0:
            # skip empty piece
            continue

        size_if_added = len(current_chunk) + len(piece)

        if  (
            size_if_added < target_size # piece still fits
            or len(current_chunk) / target_size < target_size / size_if_added # fitting it is a better compromise
        ):
            current_chunk += piece
            continue
        else:
            # piece does not fit and adding it is not a good compromise
            chunks.append(current_chunk)
            current_chunk = piece

    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks

# Example Usage

if DEBUG:
    _, sections, _ = file_to_sections(example_text)

    chunks = split_latex_text(sections[1], target_size=600)
    for i,chunk in enumerate(chunks):

        # count the tokens
        token_count = len(tiktoken.encoding_for_model("gpt-4o").encode(chunk))
        display(Markdown(f'### Chunck {i+1}/{len(chunks)}: len = {len(chunk)}, GPT tokens = {token_count}'))
        print(chunk)

In [ ]:
#@title Tokenize Latex Code (Tokens used for diff)

import re
def tokenize_latex(latex_str, only_special = False):
    """
    Tokenize a LaTeX string into words and LaTeX commands.

    Args:
        latex_str (str): The LaTeX string to tokenize.
        only_special (bool): If True, only special LaTeX token will be returned.
    Returns:
        list: A list of tokens, including words and LaTeX commands.
    """
    # Replace ` by '
    latex_str = remove_clutter_from_latex(latex_str)

    # identifiable tokens
    pattern_special = r'\\[a-zA-Z]+|[^\s\\,.;\(\){}\[\]$]+'

    # common tokens
    pattern_common = r'\$\$|\(|\)|\\\[|\\\]|\[|\]|\\.|[,.;{}$]|\s'

    if only_special:
        # give only uncommon tokens
        tokens = re.findall(pattern_special, latex_str)
    else:
        tokens = re.findall(pattern_special + '|' + pattern_common, latex_str)
    return tokens

# Example Usage

if DEBUG:
    sample_phrase = r"""
    Hey $(\trac{1}{2})$
    \[
    E = mc^2 [1,2]
    \]
    """ # This is a \ different sample sentence $\tfrac{1}{2}$ with additional text.
    tokens = tokenize_latex(sample_phrase, only_special = False)
    tokens_special = tokenize_latex(sample_phrase, only_special = True)
    print('All tokens:', tokens)
    print('Special Tokens', tokens_special)
    print('')
    print('Joined tokens:')
    print(''.join(tokens))

In [ ]:
#@title Detect tokens inside math mode

def indexes_of_math_tokens(tekens, debug = False):
    """
    Takes a list of tekens and returns the indexes of the math tokens,
    meaning that they are inside a pair of $$, $, or \[ \]
    """

    open_close = {'$':'$', '$$': '$$', '\\[': '\\]'}
    math_switches = list(open_close.keys()) + list(open_close.values())

    math_current = None # which switch keeps the math on now
    token_types = []
    for i, t in enumerate(tekens):
        if t in math_switches:
            if math_current == None:
                # no math mode yet
                if t not in open_close.keys():
                    if debug:
                        raise ValueError(f'Unmatched parenthesis! "{t}" preceeds "{open_close[t]}"! ' + str(tekens))
                else:
                    math_current = t
            else:
                # start math on already
                if t != open_close[math_current]:
                    if debug:
                        raise ValueError(f'Unmatched parenthesis! "{math_current}" preceeds "{t}"! ' + str(tekens))
                else:
                    math_current = None
            # math switch is included in that
            token_types.append(True)
        else:
            # not a math switch
            token_types.append(math_current is not None)

    return token_types

# Example Usage

if DEBUG:
    sample_phrase = r"""
    Hey $\trac{1}{2}$ and \[ E = mc^2 [1,2] \]
    """

    tokens = tokenize_latex(sample_phrase)
    is_math_mode =indexes_of_math_tokens(tokens, debug=True)
    print({t:m for t,m in zip(tokens, is_math_mode)})

In [ ]:
#@title Tokenize text, but merge all math-mode tokens

def tokenize_text_only(latex_str):
    """
    Tokenize a LaTeX string into words and LaTeX commands.

    Args:
        latex_str (str): The LaTeX string to tokenize.

    Returns:
        list: A list of tokens, including words and LaTeX commands.
        is_math_mode: A list of booleans indicating whether each token is inside math mode.
    """

    tokens = tokenize_latex(latex_str)
    math_indexes = indexes_of_math_tokens(tokens)

    tokens_text = []
    is_math_mode = []

    for token, is_math, is_math_prev in zip(tokens, math_indexes, [False] + math_indexes[:-1]):
        if is_math and is_math_prev:
            tokens_text[-1] += token
        else:
            tokens_text.append(token)
            is_math_mode.append(is_math)

    return tokens_text, is_math_mode

# Example Usage

if DEBUG:
    sample_phrase = r"""
    Hey $\trac{1}{2}$ and \[ E = mc^2 [1,2] \]
    """

    tokens_text, is_math_mode = tokenize_text_only(sample_phrase)

    print({token: is_m for token, is_m in zip(tokens_text, is_math_mode)})

In [ ]:
#@title Diff of two latex sentences.
from difflib import SequenceMatcher

def add_markdown_color(str, color, is_math_mode = False):
    """
    Add a markdown color to a string.

    Args:
        str (str): The string to color.
        color (str): The color to use.
        is_math_mode (bool): Whether the string is inside math mode.

    Returns:
        str: The colored string.
    """

    if is_math_mode:
        # inside math mode
        return r'{\color{' + color + '}' + str + '}'
    else:
        # outside of math mode
        return f' <font color="{color}"> ' + str + ' </font> '

def diff_latex_strings(str1, str2, colors = {'remove':'red', 'add':'#66BB66'}):
    """
    Get the diff between two LaTeX strings and return it as a formatted HTML string,
    highlighting whole tokens (words or LaTeX commands) that are added or removed.

    Args:
        str1 (string): The first LaTeX string
        str2 (string): The second LaTeX string

    Returns:
        diff_markdown (string): The formatted Markdown string with the diff highlighted
        is_correct (bool): True if the strings are different, False otherwise
        is_correct_text (bool): True if the strings are different outside of math mode, False otherwise

    Note: Different in term of the normalization that happens when the strings are tokenized.
    """

    tokens1, is_math_mode1 = tokenize_text_only(str1)
    tokens2, is_math_mode2 = tokenize_text_only(str2)

    matcher = SequenceMatcher(None, tokens1, tokens2)
    result = []

    is_different = False
    is_different_text = False
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':
            is_different = True

        if i1 != i2:
            if tag != 'equal':
                result.append(add_markdown_color("".join(tokens1[i1:i2]), colors['remove']))
                if not is_math_mode1[i1]:
                    is_different_text = True

        if j1 != j2:
            if (tag != 'equal'):
                result.append(add_markdown_color("".join(tokens2[j1:j2]), colors['add']))
                if not is_math_mode2[j1]:
                    is_different_text = True

        if tag == 'equal':
            result.append("".join(tokens1[i1:i2]))

    return ''.join(result), is_different, is_different_text

# Example Usage

if DEBUG:
    str1 = r"This is a sample $n=1$ sentence $\tfrac{1}{3}$ with some text \[E=mc\]"
    str2 = r"This is a different sample sentence $\tfrac{1}{2}$ with some text $$E=mc^2$$"

    diff_markdown, is_different, is_equal_different = diff_latex_strings(str1, str2)
    display(Markdown(diff_markdown))
    print(diff_markdown)
    print('is_different:', is_different)
    print('is_different_text:', is_equal_different)

In [ ]:
#@title Find the sentence/phrase closest to the provided one

import difflib

def find_closest_phrase(sample_phrase, text, length_tolerance = 5):
    """
    Find the closest phrase in the given text to the provided sample phrase.

    Args:
        text (str): The text to search in.
        sample_phrase (str): The phrase to search for.
        length_tolerance (int): The maximum length difference between the sample phrase and the found phrase.
    """

    # Tokenize the text into words
    words = tokenize_latex(text)
    sample_words = tokenize_latex(sample_phrase)
    sample_length = len(sample_words)

    sample_words_set = set(tokenize_latex(sample_phrase, only_special=True))
    sample_set_length = len(sample_words_set)

    # Function to calculate the similarity between two phrases
    def phrase_similarity(phrase1, phrase2):
        return difflib.SequenceMatcher(None, phrase1, phrase2).ratio()

    # Slide a window over the text to extract phrases of the same length as the sample
    best_similarity = 0
    best_phrase = None


    for i in range(len(words) + 1): # starting_indexes:
        for phrase_length in range(
                max(1, sample_length - length_tolerance),
                sample_length + length_tolerance
            ):

            if i + phrase_length > len(words):
                continue

            phrase_words = words[i:i + phrase_length]

            # skip if too few words intersect
            common_words = sample_words_set.intersection(set(phrase_words))
            if len(common_words) < max(1, 0.5 * sample_set_length, sample_set_length - 5):
                # failed quick check of similarity
                continue

            phrase = ''.join(phrase_words)
            similarity = phrase_similarity(phrase, sample_phrase)
            if similarity > best_similarity:
                best_similarity = similarity
                best_phrase = phrase

    if best_phrase is None:
        # raise ValueError(f'Phrase not found! \nPhrase: {sample_phrase}\n\n\nText:{text}')
        return ''

    return best_phrase

# Example Usage

if DEBUG:
    text = r"""
    This is a different $\tfrac{1}{2}$ with additional text.
    This is a.
    This is a different sample sentence with text and much more text and more text..
    """
    sample_phrase = r"This is a different sample sentence $\tfrac{1}{2}$ with additional text."

    closest_phrase = find_closest_phrase(sample_phrase, text)
    print("Closest Phrase:", closest_phrase)

In [ ]:
# from logging import exception
#@title Display correction with details

def correction_to_markdown(correction):

    original = correction['original']
    corrected = correction['corrected']
    explanation = correction['explanation']
    correction_type = correction['correction_type']
    text = correction['context']['text']
    id = correction['id']

    closest = find_closest_phrase(original, text)
    # print('Closest:', closest)

    # checks for hallucanation (original sentence not in text)
    hall_diff, is_hall, is_hall_text = diff_latex_strings(closest, original)

    # proposed correction
    corr_diff, is_corr, is_corr_text = diff_latex_strings(closest, corrected)

    # add a tab to the tittle
    tit = lambda s: f"**{s}:**" + '&hairsp;' * int(7 * (25-len(s)))


    if not is_hall:
        # did not halucinate
        details = f"""
    * Original and Corrected:

      * ```{original}```
      * ```{corrected}```
      * **id:** {id}
"""
        hall_str = ''
    else:
        # did halucinate
        details = f"""
    * Original, Closest, and Corrected:

      * ```{original}```
      * ```{closest}```
      * ```{corrected}```
      * **Hallucination:** {hall_diff}
      * **id:** {id}

</details>
"""
        # warning about halucination
        if is_hall_text:
            # hallucination is even outside of the math mode
            hall_str = ' !! '
        else:
            # hallucinated is only in the math mode
            hall_str = ' ! '

    if is_corr_text:
        # did correction outside of the math mode
        corr_str = ''
    elif is_corr:
        # did correction at least in the math mode
        corr_str = ' = '
    else:
        # did no correction at all
        corr_str = ' == '


    with_explanation = f""" * {hall_str}{corr_str} {corr_diff}
   - **{correction_type}:** {explanation}
    """

    with_details = with_explanation + details

    answer = {
        'diff': corr_diff,
        'with_explanation': with_explanation,
        'with_details': with_details,
        'hallucinated': is_hall,
        'hallucinated_text': is_hall_text,
        'corrected': is_corr,
        'corrected_text': is_corr_text,
    }

    return answer

# Example Usage

if DEBUG:
    example_correction = {
        'original': r'This is a simple text with some $\tfrac{1}{2}$ errors.',
        'corrected': 'This is a simple text with some errors.',
        'explanation': "Corrected spelling errors: 'Ths' to 'This', 'smple' to 'simple', and 'sme' to 'some'.",
        'correction_type': 'Spelling',
        'context': {'text': r'This is  a simple text with some $\tfrac{1}{2}$ errors.'},
        'id': 0
        }

    ans = correction_to_markdown(example_correction)

    print('With explanation:')
    display(Markdown(ans['with_explanation']));

    print('')
    print('With details:')
    display(Markdown(ans['with_details']));


In [ ]:
#@title OpenAI API call requesting json of corrections

def get_corrections(system:str, text:str, model:str, n:int = 1):
    """
    Summarizes the text and gets list of corrections.

    Args:
        system (str):     System prompt (instructions requesting proofreading with details)
        text (str):       Text to proofread
        model (str):            GPT model
        n (int):                Number of choices to return

    Returns:
        list:                   List of objects containing summary and a list of corrections
    """


    # dictionary of fields for each correction
    d = {
        'original':             'Complete original sentence as in text.',
        'original_meaning':     'Explanation of what the sentence states.',
        'original_errors':      'Errors in the original sentence.',
        'original_analysis':    'Grammatical analysis of the original sentence.',
        'corrected':   'Corrected sentence.',
        'explanation':          'Explanation of the correction.',
        'meaning_preservation': 'Why the corrrection preserves meaning.',
        'meaning_changes':      'Potential changes in meaning.',
        'grammar_rules':        'Grammar rules used for the correction.',
        'correction_type':      'Type of correction made.'
    }

    response = client.chat.completions.create(
        model=model,
        n=n,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": text}
        ],

        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "get_corrections",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "summary": {
                            "type": "string",
                            "description": "Summary of the supplied text (approx 10 sentences)."
                        },
                        "corrections": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    d['original']: {
                                        "type": "string",
                                        "description": "COMPLETE sentence, from the beginning to the end! EXACTLY AS IS in the supplied text!"
                                    },
                                    d['original_meaning']: {
                                        "type": "string",
                                        "descrition": "Explain the meaning of the original_sentence in its context. What does it original sentence state?"
                                    },
                                    d['original_errors']: {
                                        "type": "string",
                                        "descrition": "Detailed description of what is wrong with the original sentence."
                                    },
                                    d['original_analysis']: {
                                        "type": "string",
                                        "descrition": "Detailed grammatical analysis of the original sentence."
                                    },
                                    d['corrected']: {
                                        "type": "string",
                                        "descrition": "Corrected sentence."
                                    },
                                    d['explanation']: {
                                        "type": "string",
                                        "descrition": "Detailed reason of why do you think the correction is needed and why the corrected sentence is better than the original one."
                                    },
                                    d['meaning_preservation']: {
                                        "type": "string",
                                        "descrition": "Explain why does the corrected sentence preserve meaning in the provided context."
                                    },
                                    d['meaning_changes']: {
                                        "type": "string",
                                        "descrition": "Explain if the corrected sentence could potentially have different meaning from the original one"
                                    },
                                    d['grammar_rules']: {
                                        "type": "string",
                                        "descrition": "Detailed explanation of the grammar rules you used for the correction."
                                    },
                                    d['correction_type']: {
                                        "enum": ["Spelling", "Grammar", "Punctuation", "Sentence Structure", "Word Choice", "Concisiness", "Consistency"]
                                    },
                                },
                                "required": list(d.values()),
                                "additionalProperties": False
                            },
                        }
                    },
                    "required": ["summary", "corrections"],
                    "additionalProperties": False
                }
            }
        },
    )

    # Extract the output
    output_choices = []
    for choice in response.choices:

        # Verify the finish reason
        if choice.finish_reason != 'stop':
            raise ValueError(f"Unexpected finish reason: {choice.finish_reason}")

        # Convert content string to dictionary
        try:
            content = json.loads(choice.message.content)
        except:
            raise ValueError('Converting to json failed: json.loads()', choice)

        # Convert the keys to the concise version
        corrections = [{key: correction_raw[d[key]] for key in d.keys()}
                               for correction_raw in content['corrections']]

        # Add context to each correction
        context = {'text': text, 'summary': content['summary']}
        corrections_with_context = [{**correction, 'context': context} for correction in corrections]

        output_choices.append(corrections_with_context)

    return output_choices

# Example Usage

if DEBUG:
    system = "Proofread the provided letex text."

    text = r"""
    This is a different $\tfrac{1}{2}$ with additional text.
    This is a.
    This is a different sample sentence with text and much more text and more text..
    """

    choices = get_corrections(system, text, 'gpt-4o-mini', 2)

    id = 0
    for i, choice in enumerate(choices):
        print(f'Choice {i+1}:')
        for correction in choice:
            correction['id'] = id
            corr_markdown = correction_to_markdown(correction)
            display(Markdown(corr_markdown['with_details']))
            id += 1

        display(Markdown('---'))

# Main

Below is the actual code that needs to be run for finding typos.

In [ ]:
#@title Load latex file and display its content
#@markdown <a name="load-file"></a>

#@markdown File to revise (local file or URL):
file_path = 'https://raw.githubusercontent.com/pakocica/Proofreader/main/example.tex' #@param {type:"string"}

file_str = load_file(file_path)
display_contents(file_str)

### A Sample Scientific Papper with Multiple Errers

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

\section{Conclusion} 
Proof-reading is a vital step in the process of writting a scientific paper. 
As demonstrated, even simple mistakes can change the meaning of a sentence or a formula, leading to potential confusion. 
Ensuring the accuracy of both text and mathematical notations is crucial for the clarity and precision of scientific communication. 

In conclusion, while it may be tempting to focus solely on the content of a paper, attention to detail in its presentation is equally important. 
By avoiding common errors and carefully proofreading their work, researchers can enhance the impact of their findings and contribute more effectively to the advancement of knowledge. 

\end{document}



<IPython.core.display.Javascript object>

In [ ]:
#@title Find typos in a given section
#@markdown <a name="interactive-form"></a>

#@markdown GPT model you want to use:
gpt_model = "gpt-4o-mini" #@param ["gpt-4o", "gpt-4o-mini"]

#@markdown Section you want to work on: \
#@markdown (-1 for all sections, or 0,1,2,..., for individual sections)
i_section = -1 #@param {type:"integer"}

#@markdown Text language: \
#@markdown (edit the form to choose other languages)
language = "US English" #@param ["US English", "British English"]

#@markdown Target journal of the paper:
target_journal = "A top field Economic Theory journal" #@param {type:"string"}

#@markdown Target length (in terms of characters) of chunks the text is split into \
#@markdown * ` 2,000` – slow and details, but taking little context into considerations
#@markdown * `16,000)` – default
#@markdown * `64,000` – fast and least detailed, but taking a lot of context into consideration
target_size = 16000 #@param {type:"integer"}

#@markdown (Optional) Additional Instructions
instructions = "Fucus STRICTLY on the essential errors. Ignore the math formulas." #@param {type:"string"}

title, sections, _ = file_to_sections(file_str)

# set up the range of sections to cover
if i_section == -1:
    sections_range = range(len(sections))
else:
    sections_range = [i_section]

# Create the base widgets
tb = widgets.TabBar(['Summary', 'With Explanations', 'Debug', 'Progress'], location='top')

with tb.output_to(2, select=False):
    tb_debug = widgets.TabBar([ 'Detailed', 'Failed'], location='top')

display(Markdown('---'))

# Create the sub-widgets

with tb_debug.output_to(0, select=False):
    tb_detailed = widgets.TabBar([str(s) for s in sections_range], location='start')

with tb_debug.output_to(1, select=False):
    tb_failed = widgets.TabBar([str(s) for s in sections_range], location='start')

all_corrections = []

for i_w in [0, 1, 3]:
        with tb.output_to(i_w, select=False):
            display(Markdown(f"# {title}"))

for i_section in sections_range:
    section = sections[i_section]

    iw_section = sections_range.index(i_section)

    # inform inside the widgets about which section is processed
    if i_section >= 1:
        for i_w in [0, 1, 3]:
            with tb.output_to(i_w, select=False):
                display(Markdown(f"## Section {i_section:2}: {extract_braces_content(section)}"))
        for w in [tb_detailed, tb_failed]:
            with w.output_to(iw_section, select=False):
                display(Markdown(f"## Section {i_section:2}: {extract_braces_content(section)}"))

    # split the section into chunks
    chunks = split_latex_text(section, target_size)

    for i_chunk, text in enumerate(chunks):
        with tb.output_to(3, select=False):
            display(Markdown(f" * Chunk {i_chunk+1:2} / {len(chunks):2}"))

        system = f'''
Please proofread the latex file provided in the message.
Details:
* Language: {language}
* Target journal: {target_journal}
* Additional Instructions: {instructions}
'''

        try:
            choices = get_corrections(system, text, gpt_model)
        except ValueError as e:
            with tb.output_to(3, select=False):
                display(Markdown(' * Error: ' + str(e)))
            choices = []

        for choice in choices:
            for correction in choice:
                correction['id'] = len(all_corrections)
                cm = correction_to_markdown(correction)

                if cm['corrected_text'] and not cm['hallucinated']:
                    target_tb = tb_detailed
                    with tb.output_to(0, select=False):
                        display(Markdown(' * ' + cm['diff']))
                    with tb.output_to(1, select=False):
                        display(Markdown(cm['with_explanation']))
                else:
                    target_tb = tb_failed

                with target_tb.output_to(iw_section, select=False):
                    display(Markdown(cm['with_details']))

                # add the correction to the list of all corrections
                all_corrections.append(correction)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 *  In conclusion, while it may be tempting to focus solely on  <font color="red"> the </font>  <font color="#66BB66"> a paper's </font>  content <font color="red">  of a paper </font> , attention to <font color="red">  detail in </font>  its presentation is equally important.
   - **Concisiness:** The corrected sentence simplifies the phrasing while maintaining the original intent, resulting in improved readability.
    
    * Original and Corrected:

      * ```In conclusion, while it may be tempting to focus solely on the content of a paper, attention to detail in its presentation is equally important.```
      * ```In conclusion, while it may be tempting to focus solely on a paper's content, attention to its presentation is equally important.```
      * **id:** 18


<IPython.core.display.Javascript object>

In [ ]:
#@title Display details of specified correction
#@markdown <a name="correction-details"></a>

# def dict_to_markdown_table(data):
def dict_to_html_table(data):
    # Start the HTML table
    table = ['<table border="1" cellpadding="5" cellspacing="0" style="table-layout: fixed; width: 100%;">']

    # Add the header row
    table.append('<tr><th style="text-align:left; width:100px;">Key</th><th style="text-align:left; width:600px;">Value</th></tr>')

    # Add the data rows
    for key, value in data.items():
        if isinstance(value, str):
            table.append(f'''
                <tr>
                    <td style="word-wrap: break-word; white-space: pre-wrap;">{key}</td>
                    <td style="word-wrap: break-word; white-space: pre-wrap;">{value}</td>
                </tr>
            ''')

    # Close the HTML table
    table.append('</table>')

    return ''.join(table)

# Example usage

#@markdown Correction id:
id = 1 #@param {type:"integer"}
table = dict_to_html_table(all_corrections[id])
display(HTML(table))
# print(table)

Key,Value
original,This research paper dicusses the imprtance of detecting and correctting errors in scientifc documants.
original_meaning,"The sentence describes the aim of the research paper, which focuses on the importance of error detection and correction."
original_errors,"Multiple spelling errors exist, including 'dicusses' (should be 'discusses'), 'imprtance' (should be 'importance'), 'correctting' (should be 'correcting'), 'scientifc' (should be 'scientific'), and 'documants' (should be 'documents')."
original_analysis,The sentence is a declarative statement but is undermined by significant spelling errors.
corrected,This research paper discusses the importance of detecting and correcting errors in scientific documents.
explanation,"The corrected version fixes all the spelling errors, improving clarity and professionalism."
meaning_preservation,The meaning of the sentence remains unchanged while effectively communicating the objective of the paper.
meaning_changes,There is no change in meaning; the sentence remains focused on the same theme.
grammar_rules,"Correct spelling of English words and verb forms (discusses, correcting)."
correction_type,Spelling
